# Computational Theory 

## Problem 1 — Binary Words and Operations

Implement the SHA-256 bitwise functions: `Parity`, `Ch`, `Maj`, `Sigma0`, `Sigma1`, `sigma0`, `sigma1`.

In this problem, I implemented the core logical and bitwise functions that make up the foundation of the SHA-256 algorithm

Each function performs specific bit manipulations like rotations and XOR operations to ensure strong diffusion and unpredictability, which are key properties in cryptographic hash design.  

In this section, I implemented the six main logical and bitwise functions defined on page 10 sections 4.1.1 and 4.1.2 of the Secure Hash Standard. This is the core document for this notebook and assesment a link to it is included in the README.

The implemented functions are:
- **Ch(x, y, z)** – The “choose” function, used to select bits based on the value of *x*  
- **Maj(x, y, z)** – The “majority” function, outputs the majority bit among *x*, *y*, *z*  
- **Σ₀(x)** and **Σ₁(x)** – Perform rotations and XOR operations to achieve bit dispersion  
- **σ₀(x)** and **σ₁(x)** – Smaller versions of Σ, also involving shifts and rotations  
- **Parity(x, y, z)** – Used in SHA-1, but included here for comparison and testing  

Each function operates on **32-bit words**, using NumPy’s `uint32` type to ensure the same behavior as the specification. The formulas come directly from the standard:

\[
\begin{align*}
Ch(x, y, z) &= (x \land y) \oplus (\lnot x \land z) \\
Maj(x, y, z) &= (x \land y) \oplus (x \land z) \oplus (y \land z) \\
Σ_0(x) &= ROTR^2(x) \oplus ROTR^{13}(x) \oplus ROTR^{22}(x) \\
Σ_1(x) &= ROTR^6(x) \oplus ROTR^{11}(x) \oplus ROTR^{25}(x) \\
σ_0(x) &= ROTR^7(x) \oplus ROTR^{18}(x) \oplus SHR^3(x) \\
σ_1(x) &= ROTR^{17}(x) \oplus ROTR^{19}(x) \oplus SHR^{10}(x)
\end{align*}
\]



Below we provide:

- Clear docstrings for each function.
- Explanations of the logical behaviour (algorithmic steps)
- Tests comparing small examples and showing values in hex.


### Background Reading

While researching how cryptographic hash functions work, I came across an article called [*Cryptography Hash Method MD2 (Message Digest 2) – Step-by-Step Explanation Made Easy with Python*](https://nickthecrypt.medium.com/cryptography-hash-method-md2-message-digest-2-step-by-step-explanation-made-easy-with-python-10faa2e35e85) by Nick The Crypt. Although that article focuses on the older MD2 algorithm, it helped me understand the overall logic behind hash functions how they mix and transform bits to make outputs unpredictable. MD2 and SHA-256 are quite different in their design (MD2 uses substitution tables, while SHA-256 relies on logical operations and rotations), but the core idea of achieving diffusion and non-reversibility is the same. Reading it gave me a better appreciation for why SHA-256 uses helper functions like `Ch`, `Maj`, and the Σ/σ functions I implemented here.


### Logical Explanation – Bitwise Operations and Helper Functions

The functions below form the mathematical backbone of cryptographic hash algorithms like SHA-256.  
They rely on **bitwise operations** that manipulate the binary representation of 32-bit integers to create diffusion (mixing of input bits).

1. **_rotr(x, n):**  
   This function performs a *bitwise rotation* of a 32-bit integer `x` to the right by `n` positions.  
   Unlike a normal shift, the bits that “fall off” on the right side are wrapped around to the left side.  
   This operation preserves all bits while changing their order an important property for reversible bit mixing.

2. **Parity(x, y, z):**  
   This computes the bitwise XOR (exclusive OR) of three values.  
   Each bit in the result is 1 if an odd number of corresponding bits among `x`, `y`, and `z` are 1.  
   It’s a simple way to measure “odd parity” among bits and is used to combine data unpredictably.

3. **Ch(x, y, z):**  
   Also called the *choose* function.  
   It picks bits from `y` or `z` depending on whether the corresponding bit in `x` is 1 or 0.  
   This is a conditional operation at the bit level similar to `if (x) choose y else choose z`.

4. **Maj(x, y, z):**  
   Also known as the *majority* function.  
   For each bit position, it outputs 1 if at least two of the three inputs have that bit set.  
   This simulates a logical majority vote for every bit position.

In [2]:

import numpy as np # Import NumPy for 32-bit unsigned integer support

# Helper for rotating 32-bit integers
def _rotr(x: np.uint32, n: int) -> np.uint32:
    """Rotate-right (32-bit) helper."""
     # Shift bits right by n and wrap shifted bits from the right end to the left
    return np.uint32((x >> n) | (x << (32 - n)))

# Bitwise operations used in SHA-like algorithms
def Parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Parity(x,y,z) = x XOR y XOR z."""
     # XOR returns 1 only when an odd number of inputs have 1 in that bit position
    return np.uint32(x ^ y ^ z)

def Ch(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Ch(x,y,z) = (x AND y) XOR ((NOT x) AND z)."""
    # For each bit in x: if bit is 1 → choose y; if bit is 0 → choose z
    return np.uint32((x & y) ^ (~x & z))

def Maj(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """Maj(x,y,z) = (x AND y) XOR (x AND z) XOR (y AND z)."""
     # Returns 1 if the majority (two or more) of input bits are 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


### Logical Explanation – Sigma and sigma Functions

The Sigma functions introduce **nonlinearity** and **diffusion** into the hashing process.  
They combine **bit rotations** and **bit shifts** to ensure that small changes in the input produce large, unpredictable changes in the output a property known as the *avalanche effect*.

- **Σ0(x)** and **Σ1(x)**:  
  These are used in the *main compression function* of SHA algorithms.  
  Each function applies multiple rotate-right operations and XORs the results together.  
  This mixes bits from distant positions within the 32-bit word, increasing complexity.

- **σ0(x)** and **σ1(x):**  
  These are the *small sigma* functions, used when expanding the message schedule in SHA algorithms.  
  They use a combination of rotate-right and right-shift operations (`>>`).  
  The right shift (`SHR`) introduces zeros from the left side, losing information which helps achieve more randomness and diffusion during message expansion.


In [3]:
# Uppercase Sigma functions (used in main compression)
def Sigma0(x: np.uint32) -> np.uint32:
    """Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)."""
      # Combine rotated versions of x to achieve bit diffusion
    return np.uint32(_rotr(x, 2) ^ _rotr(x, 13) ^ _rotr(x, 22))

def Sigma1(x: np.uint32) -> np.uint32:
    """Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)."""
    # Similar to Sigma0 but uses different rotation constants
    return np.uint32(_rotr(x, 6) ^ _rotr(x, 11) ^ _rotr(x, 25))

# Lowercase sigma functions (used for message schedule)
def sigma0(x: np.uint32) -> np.uint32:
    """σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)."""
     # Use rotate-right and logical right shift to mix bits in the message schedule
    return np.uint32(_rotr(x, 7) ^ _rotr(x, 18) ^ (x >> 3))

def sigma1(x: np.uint32) -> np.uint32:
    """σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)."""
      # Another mix function for message expansion with different bit positions
    return np.uint32(_rotr(x, 17) ^ _rotr(x, 19) ^ (x >> 10))


### Testing and Verification

The cell below verifies that all previously defined functions behave as expected.  
By using fixed 32-bit hexadecimal test values for `x`, `y`, and `z`, we can visually confirm the correctness of each operation.

Each `print()` statement displays the function output in **hexadecimal format** for easy reading.  
If the outputs are consistent with the expected bitwise behaviors (e.g., XOR, AND, rotations), then the implementation is correct.

These tests act as a **sanity check** before integrating the functions into larger cryptographic processes.


In [4]:
# Quick test to make sure everything works
x = np.uint32(0x12345678)
y = np.uint32(0x9abcdef0)
z = np.uint32(0x0f1e2d3c)

# Print results of each function in hexadecimal format for clarity
print("Parity:  ", f"{Parity(x,y,z):#010x}")
print("Ch:      ", f"{Ch(x,y,z):#010x}")
print("Maj:     ", f"{Maj(x,y,z):#010x}")
print("Sigma0:  ", f"{Sigma0(x):#010x}")
print("Sigma1:  ", f"{Sigma1(x):#010x}")
print("sigma0:  ", f"{sigma0(x):#010x}")
print("sigma1:  ", f"{sigma1(x):#010x}")


Parity:   0x8796a5b4
Ch:       0x1f3e7f74
Maj:      0x1a3c5e78
Sigma0:   0x66146474
Sigma1:   0x3561abda
sigma0:   0xe7fce6ee
sigma1:   0xa1f78649


### Test Results and Explanation 

As you can see, the above results appear correct and consistent with what we’d expect from these logical and bitwise operations. Each function gives a distinct output, which shows that the rotations, shifts, and XOR/AND combinations are behaving properly.

Parity(x, y, z) gives the XOR of all three values, evenly mixing their bits.

Ch(x, y, z) acts like a conditional function, choosing bits from y or z based on the value of x.

Maj(x, y, z) produces the majority bit from the three inputs.

Sigma0 and Sigma1 apply multiple right rotations and XORs to increase bit diffusion, which is key for the non-linearity in SHA-256.

sigma0 and sigma1 use a mix of rotations and right shifts to help expand the message schedule.

## Problem 2 — Fractional Parts of Cube Roots

In this problem, we’ll calculate the SHA-256 round constants, which are derived from the fractional parts of the cube roots of the first 64 prime numbers.

These constants appear at the bottom of page 11 of the Secure Hash Standard which we mentioned in problem 1.

We’ll compute them using numpy step by step, then compare our results to the official values from the secure hash standard.

### Generate first 64 primes

In [1]:
# Function to generate the first n prime numbers 
# n in this case will be the first 64 primes 
import numpy as np

def primes(n):
    primes_list = []
    num = 2
    while len(primes_list) < n:
        for p in primes_list:
            if num % p == 0:
                break
        else:
            primes_list.append(num)
        num += 1
    return np.array(primes_list)

# Generate the first 64 primes
prime_nums = primes(64)
prime_nums


array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311])

As we can see the function above outputs the first 64 primes.

### Cube roots

Now, for each prime number, we calculate its cube root. The cube root is the number which, multiplied by itself three times, equals the original number see cell below

In [6]:
cube_roots = np.cbrt(prime_nums)  # cube root of each prime
cube_roots[:64]  # preview the first 64 cube roots

array([1.25992105, 1.44224957, 1.70997595, 1.91293118, 2.22398009,
       2.35133469, 2.57128159, 2.66840165, 2.84386698, 3.07231683,
       3.14138065, 3.33222185, 3.44821724, 3.50339806, 3.60882608,
       3.75628575, 3.89299642, 3.93649718, 4.0615481 , 4.14081775,
       4.1793392 , 4.29084043, 4.36207067, 4.4647451 , 4.59470089,
       4.65700951, 4.68754815, 4.7474594 , 4.77685618, 4.83458813,
       5.0265257 , 5.07875308, 5.15513674, 5.18010147, 5.30145919,
       5.32507402, 5.39469071, 5.46255557, 5.50687845, 5.57205466,
       5.63574079, 5.65665283, 5.75896522, 5.77899657, 5.81864787,
       5.83827246, 5.95334181, 6.06412699, 6.1001702 , 6.11803317,
       6.15344949, 6.20582179, 6.22308425, 6.30799355, 6.35786118,
       6.40695858, 6.45531481, 6.47127363, 6.51868392, 6.54991162,
       6.56541443, 6.6418522 , 6.74599671, 6.77516895])



Above, we calculated the **cube roots** of the first 64 prime numbers using NumPy’s built-in function `np.cbrt()`.  
This gives us the decimal (floating-point) representation of each cube root.

The concept is the **cube root** of a number is the value that, when multiplied by itself three times, gives the original number back.

The implemented operation is:

- **Cube Root (³√x)** – Finds the value such that **(³√x)³ = x**

Here are a few examples:

- **³√2 ≈ 1.2599**  
- **³√3 ≈ 1.4422**  
- **³√5 ≈ 1.7099**  
- **³√7 ≈ 1.9129**

These results above show the cube roots of the first 64 prime numbers.  
For the next step, we’ll extract only the **fractional part** (the digits after the decimal point), since that’s what’s used to generate the constants in the **Secure Hash Standard (SHA-256)**.




### Fractional parts of cube roots


In [7]:
fractional_parts = cube_roots % 1  # keep only the part after the decimal
fractional_parts[:64]  # preview first 64 fractional parts

array([0.25992105, 0.44224957, 0.70997595, 0.91293118, 0.22398009,
       0.35133469, 0.57128159, 0.66840165, 0.84386698, 0.07231683,
       0.14138065, 0.33222185, 0.44821724, 0.50339806, 0.60882608,
       0.75628575, 0.89299642, 0.93649718, 0.0615481 , 0.14081775,
       0.1793392 , 0.29084043, 0.36207067, 0.4647451 , 0.59470089,
       0.65700951, 0.68754815, 0.7474594 , 0.77685618, 0.83458813,
       0.0265257 , 0.07875308, 0.15513674, 0.18010147, 0.30145919,
       0.32507402, 0.39469071, 0.46255557, 0.50687845, 0.57205466,
       0.63574079, 0.65665283, 0.75896522, 0.77899657, 0.81864787,
       0.83827246, 0.95334181, 0.06412699, 0.1001702 , 0.11803317,
       0.15344949, 0.20582179, 0.22308425, 0.30799355, 0.35786118,
       0.40695858, 0.45531481, 0.47127363, 0.51868392, 0.54991162,
       0.56541443, 0.6418522 , 0.74599671, 0.77516895])

Above we remove the whole number part and keep only the decimals. For example, if a cube root is 5.874, we only keep 0.874. This is the part that will be turned into constants. 

### Conversion to match constants

Now that we have the **fractional part** of each cube root, we need to turn it into the **constants used in SHA-256**.  

Here’s what we are doing in simple terms:

1. **Multiply by 2³²**  
   - The fractional part is a number between 0 and 1 (e.g., 0.259921).  
   - Multiplying by 2³² scales it to a **32-bit integer range**, because SHA-256 constants are **32-bit words**.

2. **Convert to 32-bit unsigned integers**  
   - This truncates any extra decimal part and ensures the value fits exactly into 32 bits.  
   - We use `np.uint32` to match the format required by the SHA-256 standard.

3. **Convert to hexadecimal format**  
   - SHA-256 constants are written in hexadecimal (base 16).  
   - Using `hex()` turns our integers into hex strings, which we can then compare to the **official constants** from the Secure Hash Standard.

**Example:**  
If the fractional part is `0.259921`, then we scale it to a 32-bit integer by multiplying by 2^32:

0.259921 × 2^32 ≈ 1116352408

Converting this integer to hexadecimal gives:

1116352408 → 0x428a2f98

This matches the first constant in the SHA-256 standard.  

By repeating this for all 64 primes, we generate the **full set of SHA-256 constants**.

In [8]:
# Multiply by 2^32 and convert to 32-bit unsigned integers
constants = (fractional_parts * (2**32)).astype(np.uint32)
hex_constants = [hex(c) for c in constants]
hex_constants[:64]  # preview first 64 constants

['0x428a2f98',
 '0x71374491',
 '0xb5c0fbcf',
 '0xe9b5dba5',
 '0x3956c25b',
 '0x59f111f1',
 '0x923f82a4',
 '0xab1c5ed5',
 '0xd807aa98',
 '0x12835b01',
 '0x243185be',
 '0x550c7dc3',
 '0x72be5d74',
 '0x80deb1fe',
 '0x9bdc06a7',
 '0xc19bf174',
 '0xe49b69c1',
 '0xefbe4786',
 '0xfc19dc6',
 '0x240ca1cc',
 '0x2de92c6f',
 '0x4a7484aa',
 '0x5cb0a9dc',
 '0x76f988da',
 '0x983e5152',
 '0xa831c66d',
 '0xb00327c8',
 '0xbf597fc7',
 '0xc6e00bf3',
 '0xd5a79147',
 '0x6ca6351',
 '0x14292967',
 '0x27b70a85',
 '0x2e1b2138',
 '0x4d2c6dfc',
 '0x53380d13',
 '0x650a7354',
 '0x766a0abb',
 '0x81c2c92e',
 '0x92722c85',
 '0xa2bfe8a1',
 '0xa81a664b',
 '0xc24b8b70',
 '0xc76c51a3',
 '0xd192e819',
 '0xd6990624',
 '0xf40e3585',
 '0x106aa070',
 '0x19a4c116',
 '0x1e376c08',
 '0x2748774c',
 '0x34b0bcb5',
 '0x391c0cb3',
 '0x4ed8aa4a',
 '0x5b9cca4f',
 '0x682e6ff3',
 '0x748f82ee',
 '0x78a5636f',
 '0x84c87814',
 '0x8cc70208',
 '0x90befffa',
 '0xa4506ceb',
 '0xbef9a3f7',
 '0xc67178f2']

### Official constants 

Below we see the official SHA-256 constants. We check to see how many of our converted 64 primes match and print the result.

In [9]:
official_constants_hex = [
    "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
    "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
    "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
    "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
    "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
    "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
    "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
    "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"
]

matches = [our == off for our, off in zip(hex_constants, official_constants_hex)]
sum(matches), len(matches)  # how many match out of 64


(0, 64)

## Problem 3 — Message Padding for SHA-256 

In this problem, we write a generator function **`block_parse(msg)`** that takes a message (as a `bytes` object) and outputs **512-bit (64-byte) blocks**, exactly the way SHA-256 expects them.  
Because SHA-256 can *only* process messages in fixed-size 512-bit chunks, we must follow the **official padding rules** from the Secure Hash Standard.

Before writing any code, it helps to break down the padding process into simple steps with clear equations.

In simplest terms Imagine you have a message of different lengths, but SHA-256 works best when the message is a certain size (512 bits). So, if your message is too short, you add some extra bits to make it the right size. Think of it like adding extra spaces at the end of a sentence to fill a whole page. https://medium.com/@bajrang1081siyag/how-sha-256-works-4951088ab9f8

---

### Why padding is important
SHA-256 processes data in chunks of:

**512 bits = 64 bytes**

But almost no real message has a size that is exactly a multiple of 64 bytes.  
To handle any message length, SHA-256 forces the padded message to satisfy:

**padded_length_bits ≡ 0 (mod 512)**

and also appends the original message length at the end:

**L = 8 × message_length_bytes**

This value is encoded as a 64-bit (8-byte) big-endian integer.

---

### How SHA-256 Padding Works (With Visuals + Equations)

Let your original message be:

**M = bytes(m₀, m₁, …, mₙ₋₁)**  
**L = 8 × n** (its bit-length)

Padding happens in three steps:

---

#### **Step 1 — Append a “1” bit**

In hex, this is:

**0x80 → 10000000₂**

---

#### **Step 2 — Append zero bytes until the length ≡ 56 (mod 64)**

We keep adding:

**0x00**

until:

**current_length_bytes ≡ 56 (mod 64)**

The final 8 bytes of the block are reserved for the length field.

---

#### **Step 3 — Append the original length L as a 64-bit big-endian integer**

For example, for the message `"abc"`:

**n = 3**  
**L = 8 × 3 = 24 bits**  
**L = 0x0000000000000018**

We stop here for this problem, as our goal was simply to produce the correctly padded 512-bit blocks according to the SHA-256 standard. While the full hashing process involves message scheduling, round constants, and 64 compression steps to produce the final digest, that's beyond the scope of this task. Down the line, we could extend this generator to feed into a compression function or even build a complete SHA-256 hasher






In [10]:
# Helper: convert an integer bit length into an 8-byte big-endian representation.
def length_to_64bit_big_endian(bit_length: int) -> bytes:
    """
    Convert `bit_length` (an int) to an 8-byte (64-bit) big-endian bytes object.

    Example:
      bit_length = 8  -> b'\x00\x00\x00\x00\x00\x00\x00\x08'
    """
    return bit_length.to_bytes(8, byteorder='big')


### Why we need the helper

SHA-256 stores the original message length in the final 64 bits (8 bytes) of the padded message.  
This helper turns the integer `len(msg) * 8` (bits) into the exact 8-byte big-endian representation required by the standard.

In [11]:
def block_parse(msg: bytes):
    """
    Generator yielding 512-bit (64-byte) blocks padded per SHA-256 rules.

    Args:
      msg (bytes): original message bytes

    Yields:
      bytes: successive 64-byte blocks (the final block(s) include padding)
    """
    # 1) Original message length in bits
    bit_len = len(msg) * 8

    # 2) Start padding by appending 0x80 (1000 0000 in bits)
    padded = msg + b"\x80"

    # 3) Append zero bytes until the length mod 64 equals 56.
    #    That leaves 8 bytes at the end for the 64-bit length field.
    while len(padded) % 64 != 56:
        padded += b"\x00"

    # 4) Append the 64-bit big-endian length field
    padded += length_to_64bit_big_endian(bit_len)

    # 5) Yield each 64-byte block
    for i in range(0, len(padded), 64):
        yield padded[i:i+64]


### Explanation of `block_parse`

- Above we compute `bit_len = len(msg) * 8` because the standard records **bits**, not bytes.  
- `0x80` is appended to mark the start of padding (a single `1` bit followed by zeros).  
- We add zero bytes until `len(padded) % 64 == 56`. Why 56? Because the final 8 bytes are reserved for the 64-bit message length (56 + 8 = 64 bytes).  
- Finally, we append the 8-byte length. At this point the total padded message length is a multiple of 64 bytes, so we can yield fixed 64-byte blocks.


In [12]:
# Nicely print blocks for inspection
def print_blocks(label: str, msg: bytes, max_bytes_preview: int = 16):
    """
    Print each block as hex and show the first `max_bytes_preview` Also prints block length to confirm 64 bytes.
    """
    print(f"\n=== {label} ===")
    for i, block in enumerate(block_parse(msg)):
        # Show first few bytes as hex to keep output compact
        preview = block[:max_bytes_preview].hex()
        print(f"Block {i}: len={len(block)} bytes, preview (first {max_bytes_preview} bytes): {preview}")


### Tests

We'll test multiple message lengths to observe padding behavior:

1. Empty message (`b""`) — should produce exactly one block containing padding + length.
2. Short message like `b"abc"` — typical small message case.
3. Message of exactly 56 bytes — edge case: after appending 0x80 it will exceed 56 mod 64, so it should produce two blocks.
4. Message of 60 bytes — another edge-case that triggers two final blocks.
5. Message longer than 64 bytes (e.g. 80 bytes) — tests multi-block messages with padding appended after the last full block.


In [13]:
# Test cases
print_blocks("Test 1: Empty message", b"")
print_blocks("Test 2: 'abc'", b"abc")
print_blocks("Test 3: 56 bytes ('A'*56)", b"A" * 56)
print_blocks("Test 4: 60 bytes ('B'*60)", b"B" * 60)
print_blocks("Test 5: 80 bytes ('C'*80)", b"C" * 80)



=== Test 1: Empty message ===
Block 0: len=64 bytes, preview (first 16 bytes): 80000000000000000000000000000000

=== Test 2: 'abc' ===
Block 0: len=64 bytes, preview (first 16 bytes): 61626380000000000000000000000000

=== Test 3: 56 bytes ('A'*56) ===
Block 0: len=64 bytes, preview (first 16 bytes): 41414141414141414141414141414141
Block 1: len=64 bytes, preview (first 16 bytes): 00000000000000000000000000000000

=== Test 4: 60 bytes ('B'*60) ===
Block 0: len=64 bytes, preview (first 16 bytes): 42424242424242424242424242424242
Block 1: len=64 bytes, preview (first 16 bytes): 00000000000000000000000000000000

=== Test 5: 80 bytes ('C'*80) ===
Block 0: len=64 bytes, preview (first 16 bytes): 43434343434343434343434343434343
Block 1: len=64 bytes, preview (first 16 bytes): 43434343434343434343434343434343


### Test Outputs conclusion


#### **Test 1 — Empty Message (`b""`)**

**First bytes:** `80 00 00 00 00 00 00 00 ...`

- Since the message is empty, the first byte is `0x80`.  
- The final 8 bytes encode the original length (which is 0 bits).  
- Everything fits into a single 64-byte block.

---

#### **Test 2 — `"abc"`**

**First bytes:** `61 62 63 80 00 00 00 00 ...`

- `"abc"` converts to `61 62 63`.  
- Then comes the `0x80` padding byte.  
- The final 8 bytes represent **24 bits** (the length of `"abc"`).  
- Again, all fits into one block.

---

#### **Test 3 — 56 Bytes of `'A'`**

**Block 0 preview:** all `41` bytes  
**Block 1 preview:** all zeros (until the final length field)

Block 0 is completely filled by the message itself:

- There is **no room** for `0x80` or the 8-byte length field.  
- So SHA-256 creates a **second block** that contains the padding and length.  

Two blocks are expected here, which show the result is correct.

---

#### **Test 4 — 60 Bytes of `'B'`**

**Block 0 preview:** all `42` bytes  
**Block 1 preview:** zeros (until padding + length)

This is another overflow case:

- After adding `0x80`, Block 0 would exceed the 56-byte padding point.  
- Therefore a second block is needed for the zero padding and length field.

This matches what the standard says should happen.

---

#### **Test 5 — 80 Bytes of `'C'`**

**Block 0 preview:** first 64 bytes of `'C'`  
**Block 1 preview:** next 16 bytes of `'C'`

- The first 64 bytes fill Block 0.  
- Block 1 starts with the remaining 16 bytes.  
- Padding and the final 8-byte length field are appended at the end of Block 1.

Two blocks are expected here and the output matches this.

---

#### Final Summary


| Test | Input Size | Expected Blocks | Result |
|------|------------|----------------|--------|
| 1 | 0 bytes | 1 | Correct |
| 2 | 3 bytes | 1 | Correct |
| 3 | 56 bytes | 2 | Correct |
| 4 | 60 bytes | 2 | Correct |
| 5 | 80 bytes | 2 | Correct |




## Problem 4 — Hash Computation

In this problem, we'll implement the core SHA-256 hash computation function that brings together everything we've built in Problems 1-3.

**What we're building:**
- A `hash(current, block)` function that processes a single 512-bit block
- This follows Section 6.2.2 "SHA-256 Hash Computation" on page 22 of the Secure Hash Standard
- We'll also generate the initial hash values (H₀) from square roots of primes, similar to how we generated round constants in Problem 2 this creates continuity rather than having manual imput.

**How everything connects:**

Reaching this point I think it is a good idea to refelect on the previous problems.

- **Problem 1** gave us the bitwise functions (Ch, Maj, Σ, σ) needed for compression
- **Problem 2** gave us the 64 round constants (K) derived from cube roots
- **Problem 3** gave us properly padded 512-bit blocks to process
- **Problem 4** uses all of these to compute the actual hash values

The hash function performs 64 rounds of operations on each block, mixing the data with constants and previous hash values to produce a new hash state. This is the heart of SHA-256's security small changes in input produce completely different outputs.

For background on how SHA-256's compression function works, this visual tool [Understanding SHA-256: A Visual Guide](https://sha256algorithm.com/) provides helpful visualizations of the process we're implementing here.

In [19]:
# Generate the initial hash values from square roots of first 8 primes
def generate_initial_hash():
    """
    Generate the 8 initial hash values (H0-H7) for SHA-256.
    These come from the fractional parts of square roots of the first 8 primes.
    """
    # Get first 8 primes (reusing our primes function from Problem 2)
    first_8_primes = primes(8)
    print(f"First 8 primes: {first_8_primes}")
    
    # Calculate square roots
    square_roots = np.sqrt(first_8_primes)
    print(f"\nSquare roots: {square_roots}")
    
    # Extract fractional parts
    fractional_parts = square_roots % 1
    print(f"\nFractional parts: {fractional_parts}")
    
    # Scale to 32-bit integers
    initial_hash = (fractional_parts * (2**32)).astype(np.uint32)
    
    # Display in hex (the standard format)
    print(f"\nInitial hash values (H0-H7) in hex:")
    for i, h in enumerate(initial_hash):
        print(f"  H{i} = {h:#010x}")
    
    return initial_hash

# Generate and store the initial hash values
H_INITIAL = generate_initial_hash()

# Also verify against official SHA-256 initial values
official_H0 = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
               0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

print(f"\n{'='*60}")
print("Verification against official SHA-256 initial values:")
print(f"{'='*60}")
all_match = True
for i, (ours, official) in enumerate(zip(H_INITIAL, official_H0)):
    match = "✓" if ours == official else "✗"
    print(f"H{i}: {ours:#010x} vs {official:#010x} {match}")
    if ours != official:
        all_match = False

if all_match:
    print("\n✓ All initial hash values match the SHA-256 standard!")
else:
    print("\n✗ Warning: Initial values don't match standard")

First 8 primes: [ 2  3  5  7 11 13 17 19]

Square roots: [1.41421356 1.73205081 2.23606798 2.64575131 3.31662479 3.60555128
 4.12310563 4.35889894]

Fractional parts: [0.41421356 0.73205081 0.23606798 0.64575131 0.31662479 0.60555128
 0.12310563 0.35889894]

Initial hash values (H0-H7) in hex:
  H0 = 0x6a09e667
  H1 = 0xbb67ae85
  H2 = 0x3c6ef372
  H3 = 0xa54ff53a
  H4 = 0x510e527f
  H5 = 0x9b05688c
  H6 = 0x1f83d9ab
  H7 = 0x5be0cd19

Verification against official SHA-256 initial values:
H0: 0x6a09e667 vs 0x6a09e667 ✓
H1: 0xbb67ae85 vs 0xbb67ae85 ✓
H2: 0x3c6ef372 vs 0x3c6ef372 ✓
H3: 0xa54ff53a vs 0xa54ff53a ✓
H4: 0x510e527f vs 0x510e527f ✓
H5: 0x9b05688c vs 0x9b05688c ✓
H6: 0x1f83d9ab vs 0x1f83d9ab ✓
H7: 0x5be0cd19 vs 0x5be0cd19 ✓

✓ All initial hash values match the SHA-256 standard!


### The Hash Computation Function

Now we implement `hash(current, block)`, which takes:
- `current`: The current hash state (8 × 32-bit words)
- `block`: A 512-bit (64-byte) message block from Problem 3

**What happens inside:**

**Step 1 — Message Schedule (W)**
- Break the 512-bit block into 16 × 32-bit words (W₀ to W₁₅)
- Extend to 64 words (W₁₆ to W₆₃) using the **σ functions from Problem 1**
- Formula: Wₜ = σ₁(Wₜ₋₂) + Wₜ₋₇ + σ₀(Wₜ₋₁₅) + Wₜ₋₁₆

**Step 2 — Initialize Working Variables**
- Set a, b, c, d, e, f, g, h to the current hash values

**Step 3 — 64 Rounds of Compression**
- For each round t (0 to 63):
  - T₁ = h + Σ₁(e) + Ch(e,f,g) + Kₜ + Wₜ ← Uses **Problem 1 functions**
  - T₂ = Σ₀(a) + Maj(a,b,c) ← Uses **Problem 1 functions**
  - Shift all variables: h=g, g=f, f=e, e=d+T₁, d=c, c=b, b=a, a=T₁+T₂
  - Uses **round constants from Problem 2** (Kₜ)

**Step 4 — Add to Current Hash**
- Add each working variable back to the corresponding hash value
- This becomes the new hash state for the next block

In [55]:
import warnings

def hash(current: np.ndarray, block: bytes) -> np.ndarray:
    """
    Compute the next hash value given current hash and a message block.
    
    Args:
        current: array of 8 uint32 values (current hash state H0-H7)
        block: exactly 64 bytes (512 bits) from block_parse()
    
    Returns:
        array of 8 uint32 values (new hash state)
    """
    # Ensure block is exactly 64 bytes
    assert len(block) == 64, f"Block must be 64 bytes, got {len(block)}"

    # Suppress overflow warnings - they're expected in SHA-256's modular arithmetic
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        
        # ========================================
        # STEP 1: Prepare Message Schedule (W)
        # ========================================
        # Parse block into 16 big-endian 32-bit words
        W = np.zeros(64, dtype=np.uint32)
        for t in range(16):
            # Extract 4 bytes and convert from big-endian
            W[t] = np.uint32(int.from_bytes(block[t*4:(t+1)*4], byteorder='big'))
        
        # Extend to 64 words using sigma functions from Problem 1
        for t in range(16, 64):
            W[t] = np.uint32(sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16])
        
        # ========================================
        # STEP 2: Initialize Working Variables
        # ========================================
        a, b, c, d, e, f, g, h = current
        
        # ========================================
        # STEP 3: 64 Rounds of Compression
        # ========================================
        for t in range(64):
            # T1 uses: h, Sigma1, Ch (from Problem 1), K constant (from Problem 2), W
            T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + constants[t] + W[t])
            
            # T2 uses: Sigma0, Maj (from Problem 1)
            T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
            
            # Update working variables (shift and mix)
            h = g
            g = f
            f = e
            e = np.uint32(d + T1)
            d = c
            c = b
            b = a
            a = np.uint32(T1 + T2)
        
        # ========================================
        # STEP 4: Compute New Hash Value
        # ========================================
        # Add compressed values back to current hash
        new_hash = np.array([
            np.uint32(current[0] + a),
            np.uint32(current[1] + b),
            np.uint32(current[2] + c),
            np.uint32(current[3] + d),
            np.uint32(current[4] + e),
            np.uint32(current[5] + f),
            np.uint32(current[6] + g),
            np.uint32(current[7] + h)
        ], dtype=np.uint32)
    
    return new_hash


### Complete SHA-256 Implementation

Now we can write a complete `sha256(message)` function that ties everything together:

1. **Padding** — Use `block_parse()` from Problem 3 to get 512-bit blocks
2. **Initialize** — Start with H_INITIAL values we just generated
3. **Process** — Feed each block through our `hash()` function
4. **Output** — Concatenate the final 8 hash values into a 256-bit (32-byte) hex string



In [21]:
def sha256(message: bytes) -> str:
    """
    Complete SHA-256 hash function.
    
    Args:
        message: input bytes to hash
    
    Returns:
        64-character hex string (the SHA-256 digest)
    """
    # Start with initial hash values
    current_hash = H_INITIAL.copy()
    
    # Process each 512-bit block from Problem 3's block_parse
    for block in block_parse(message):
        current_hash = hash(current_hash, block)
    
    # Convert final hash to hex string
    result = ''.join(f'{h:08x}' for h in current_hash)
    return result

### Testing Our SHA-256 Implementation

Let's test our implementation with known test vectors from the SHA-256 standard and compare them to Python's built-in `hashlib` to verify correctness.

We'll test:
1. Empty message
2. "abc" (the classic test case)
3. A longer message
4. A message that requires multiple blocks

In [56]:
import hashlib

def test_sha256(message_str: str):
    """
    Test our SHA-256 against Python's hashlib implementation.
    """
    message = message_str.encode('utf-8')
    
    # Our implementation
    our_hash = sha256(message)
    
    # Python's built-in
    expected_hash = hashlib.sha256(message).hexdigest()
    
    # Check if they match
    match = "✓ PASS" if our_hash == expected_hash else "✗ FAIL"
    
    print(f"\nMessage: {repr(message_str)}")
    print(f"  Length: {len(message)} bytes")
    print(f"  Our hash:      {our_hash}")
    print(f"  Expected hash: {expected_hash}")
    print(f"  {match}")
    
    return our_hash == expected_hash

# Run test cases
print("=" * 70)
print("SHA-256 Implementation Tests")
print("=" * 70)

test1 = test_sha256("")  # Empty message
test2 = test_sha256("abc")  # Classic test case
test3 = test_sha256("The quick brown fox jumps over the lazy dog")  # Longer message
test4 = test_sha256("a" * 100)  # Multiple blocks needed

print("\n" + "=" * 70)
if all([test1, test2, test3, test4]):
    print("✓ ALL TESTS PASSED!")
else:
    print("✗ Some tests failed")
print("=" * 70)

SHA-256 Implementation Tests

Message: ''
  Length: 0 bytes
  Our hash:      e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
  Expected hash: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
  ✓ PASS

Message: 'abc'
  Length: 3 bytes
  Our hash:      ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
  Expected hash: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
  ✓ PASS

Message: 'The quick brown fox jumps over the lazy dog'
  Length: 43 bytes
  Our hash:      d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592
  Expected hash: d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592
  ✓ PASS

Message: 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
  Length: 100 bytes
  Our hash:      2816597888e4a0d3a36b82b83316ab32680eb8f00f8cd3b904d681246d285a0e
  Expected hash: 2816597888e4a0d3a36b82b83316ab32680eb8f00f8cd3b904d681246d285a0e
  ✓ PASS

✓ ALL

## Conclusion

**How Everything Tied Together:**

1. **Problem 1 (Bitwise Operations)** — We built the molecular-level building blocks: Ch, Maj, and the Σ/σ functions. These simple operations on individual bits create the foundation for cryptographic strength. Think of these as the basics.

2. **Problem 2 (Round Constants)** — We generated the 64 constants from cube roots of primes. By deriving constants from well-known mathematical values, the SHA-256 designers proved these weren't chosen to hide backdoors. Anyone can verify them.

3. **Problem 3 (Message Padding)** — We learned how to prepare any message for processing by breaking it into uniform 512-bit blocks. This padding scheme ensures that even tiny messages get properly mixed, and the length encoding prevents length-extension attacks.

4. **Problem 4 (Hash Computation)** — We brought it all together: the bitwise functions manipulate data, the round constants add mathematical variety, the padded blocks feed in sequentially, and 64 rounds of compression create a final digest that's practically impossible to reverse.

**The elegance of SHA-256's Design:**

What makes SHA-256 elegant is how **simple components create complex security**. Each individual piece (a rotation, an XOR, a majority function) is trivial to compute, yet when combined through 64 rounds across multiple blocks, they create a one-way function with remarkable properties:

- **Avalanche effect**: Flip one input bit → approximately 50% of output bits flip
- **Collision resistance**: Finding two messages with the same hash requires ~2¹²⁸ attempts
- **Pre-image resistance**: Given a hash, finding the original message requires ~2²⁵⁶ attempts

### Interesting Research and Alternative Approaches

**1. Hardware Acceleration**
Modern CPUs include SHA extensions (Intel SHA-NI, ARM Crypto Extensions) that compute SHA-256 blocks in just a few clock cycles. Projects like [Bitcoin mining ASICs](https://en.bitcoin.it/wiki/Mining_hardware_comparison) pushed SHA-256 hardware to extreme efficiency modern miners compute trillions of hashes per second.

**2. Parallel SHA-256**
Because each 512-bit block depends on the previous hash output, SHA-256 seems inherently sequential. However, researchers have developed clever tree-based approaches for hashing large files in parallel. The idea: split the file into chunks, hash them independently, then hash the results together in a Merkle tree structure. This is used in systems like [Git's object storage](https://git-scm.com/book/en/v2/Git-Internals-Git-Objects) and blockchain technologies.

**3. Quantum Resistance**
While SHA-256 is vulnerable to Grover's algorithm (reducing security from 2²⁵⁶ to 2¹²⁸ operations on a quantum computer), it's still considered quantum-resistant for hash functions since 2¹²⁸ remains infeasible. Research into post-quantum signatures often still relies on SHA-256 as a building block. See NIST's [Post-Quantum Cryptography Standardization](https://csrc.nist.gov/projects/post-quantum-cryptography) project.

**4. Differential Cryptanalysis**
Researchers continuously probe SHA-256 for weaknesses. While full SHA-256 (64 rounds) remains secure, reduced-round versions have been attacked. In 2008, researchers broke 24-round SHA-256, demonstrating why those 64 rounds matter. This ongoing research validates the design minor changes would weaken it significantly. [Cryptanalysis of SHA-2 (Wikipedia)](https://en.wikipedia.org/wiki/SHA-2#Cryptanalysis_and_validation).

**5. Interesting Implementations**
People have implemented SHA-256 in unexpected places:
- **In SQL queries** — [computing SHA-256 purely with SQL](https://stackoverflow.com/questions/6713725/is-there-a-sha-256-implementation-in-sql) using bitwise operators
- **In Excel** — Spreadsheet formulas can compute SHA-256, though it's very slow
- **In Conway's Game of Life** — Theoretically possible since Game of Life is Turing-complete
- **As a physical puzzle** — [SHA-256 solved by hand](https://www.righto.com/2014/09/mining-bitcoin-with-pencil-and-paper.html) by Ken Shirriff, who computed a Bitcoin hash manually (took 16 minutes per hash)

### Why This Matters

Understanding SHA-256 from first principles did more for me than teach cryptography, it revealed how **complexity emerges from simplicity**. The same patterns appear everywhere in computer science: simple rules, repeated many times, create powerful emergent behaviors.

From blockchain consensus mechanisms to password storage to Git version control, SHA-256 secures much of our digital infrastructure. By building it ourselves, we've demystified one of computing's most important black boxes.

**Further Reading:**
- [How SHA-256 Works Step-By-Step](https://blog.boot.dev/cryptography/how-sha-2-works-step-by-step-sha-256/) — Visual walkthrough
- [Mining Bitcoin with Pencil and Paper](https://www.righto.com/2014/09/mining-bitcoin-with-pencil-and-paper.html) — Ken Shirriff's manual SHA-256 computation

---
